Dataset= https://www.kaggle.com/datasets/kazanova/sentiment140

In [1]:
# 1. Gerekli kütüphaneleri içe aktar
import pandas as pd                # Veri analizi ve tablo işlemleri
import re                         # Regex ile metin temizliği için
import nltk                       # Doğal dil işleme için
from sklearn.model_selection import train_test_split   # Eğitim-test ayırma
from sklearn.feature_extraction.text import TfidfVectorizer  # Metni sayıya çevirme
from sklearn.linear_model import LogisticRegression    # Makine öğrenmesi modeli
from sklearn.metrics import accuracy_score, classification_report  # Başarı metrikleri

In [2]:
# 2. nltk kütüphanesinden gerekli verileri indir
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...


True

stopwords: İngilizce'de anlam taşımayan sık kelimeleri getirir ("the", "is", "at" gibi)

lemmatizer: Kelimeleri kök forma indirger (örneğin: playing → play)

In [3]:

from nltk.corpus import stopwords                     # İngilizce durak kelimeler (the, and, etc.)
from nltk.stem import WordNetLemmatizer               # Kelimeleri kök haline getirme (örn: playing → play)

In [4]:
# 3. Veriyi oku (Kaggle CSV dosyasını indirip aynı klasöre koymalısın)
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin-1', header=None)

# 4. Sütun adlarını tanımla (Kaggle’daki bu dosya sütun ismi içermez)
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']


In [5]:
df

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


Sütun Adı ----> Açıklaması
target-->Tweet’in duygusunu (etiketini) gösterir.
🔹 0 = Negatif duygu	
🔹 4 = Pozitif duygu	
(Nötr sınıf bu veri setinde yoktur.)
id --> Tweet’in benzersiz kimliği (sayısal ID) – genellikle analizde kullanılmaz. 

date --> Tweet’in atıldığı tarih ve saat bilgisi. |

flag-->Veri toplama sırasında kullanılan filtre bilgisi. Çoğunlukla "NO_QUERY" şeklindedir. Genellikle önemsizdir. |

user-->Tweet’i atan kullanıcının kullanıcı adı. |

text-->Tweet’in kendisi.

In [6]:
# 5. Sadece target ve text sütunlarıyla ilgileniyoruz
df = df[['target', 'text']]

# 6. Target sütununda 0 = negatif, 4 = pozitif
# 2 olan nötr verileri kullanmıyoruz, bu veride zaten yok
df = df[df['target'].isin([0, 4])]

# 7. Etiketleri sadeleştir (0 = 0, 4 = 1)
df['label'] = df['target'].map({0: 0, 4: 1})


* read_csv-->Kaggle dosyasını okur (sütun başlığı olmadığından header=None)
* df.columns = [...]-->Sütunlara anlamlı isim verir
* [['target', 'text']]-->Sadece etiket ve metin sütunlarını alır
* df[df['target'].isin([0,4])]-->Sadece pozitif (4) ve negatif (0) tweet'leri seçer
* map({0:0, 4:1})-->0 → 0 (negatif), 4 → 1 (pozitif) olarak sadeleştirir

In [7]:
# 8. Metin ön işleme için fonksiyon tanımla
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = re.sub(r"http\S+|@\S+|#[A-Za-z0-9_]+", "", text)  # Link, mention ve hashtag'leri temizle
    text = re.sub(r"[^a-zA-Z ]", "", text).lower().split()  # Harf dışı karakterleri sil, küçük harfe çevir
    text = [lemmatizer.lemmatize(word) for word in text if word not in stop_words]  # Stopword'leri çıkar ve kök halini al
    return " ".join(text)

# 9. Tweet metinlerini temizle ve yeni sütuna yaz
df['clean_text'] = df['text'].apply(preprocess)


* re.sub(...)-->Tweet içindeki linkleri, etiketleri, sembolleri temizler
* lower().split()-->Küçük harfe çevirip kelimelere ayırır
* if w not in stop_words-->Gereksiz kelimeleri siler (örnek: the, and, is)
* lemmatize-->Kelimeleri kök haline getirir (örnek: loved → love)

In [8]:
# 10. Giriş (X) ve hedef (y) verilerini ayır
X = df['clean_text']   # Metinler (bağımsız değişken)
y = df['label']        # Etiketler (bağımlı değişken: 0 veya 1)

# 11. Eğitim ve test verisine böl (%80 eğitim, %20 test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


* X: Giriş verisi → tweet’in temiz metni

* y: Çıkış/etiket → pozitif mi negatif mi?

* train_test_split: Veriyi %80 eğitim, %20 test olacak şekilde böler

* random_state: Aynı sonucu alabilmek için sabitlik sağlar

In [9]:
# 12. TF-IDF ile metni sayıya çevir (model bunu anlamaz)
tfidf = TfidfVectorizer(max_features=5000)  # En çok geçen 5000 kelimeyle sınırlıyoruz
X_train_vec = tfidf.fit_transform(X_train)  # Eğitim verisi üzerinden öğren ve dönüştür
X_test_vec = tfidf.transform(X_test)        # Test verisini de dönüştür


* TF-IDF: Kelimelerin önem derecesine göre ağırlık verir

* fit_transform: Eğitim verisini öğrenip sayılara çevirir

* transform: Test verisini de aynı kurallarla sayıya çevirir

* max_features=5000: En çok kullanılan 5000 kelime ile sınırlı tutar

In [10]:
# 13. Modeli oluştur ve eğit
model = LogisticRegression()         # Basit ama etkili bir sınıflandırıcı
model.fit(X_train_vec, y_train)      # Eğitimi yap


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

* LogisticRegression(): Basit, hızlı ve etkili bir sınıflandırma modeli

* .fit(...): Modeli eğitim verisiyle eğitir (tweet → pozitif/negatif ilişkisi öğrenilir)

In [11]:
# 14. Test verisiyle tahmin yap
y_pred = model.predict(X_test_vec)


In [12]:
X_test_vec

<320000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 1884522 stored elements in Compressed Sparse Row format>

In [13]:
# 15. Modelin doğruluğunu ve raporunu yazdır
print("Doğruluk Oranı:", accuracy_score(y_test, y_pred))  # Genel başarı oranı
print("\nSınıflandırma Raporu:\n", classification_report(y_test, y_pred, target_names=["Negatif", "Pozitif"]))


Doğruluk Oranı: 0.773496875

Sınıflandırma Raporu:
               precision    recall  f1-score   support

     Negatif       0.78      0.75      0.77    159494
     Pozitif       0.76      0.79      0.78    160506

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000



* predict(...): Test verisi için pozitif mi negatif mi tahmin eder

* accuracy_score: Doğru tahmin yüzdesini hesaplar

* classification_report: Precision, recall, F1-score gibi metrikleri verir
(her sınıf için ayrı ayrı)

In [15]:
# 1. İlk 10 test verisini ham metin olarak alalım
orijinal_tweetler = X_test[:10].reset_index(drop=True)

# 2. Gerçek etiketler ve modelin tahminlerini resetleyerek al
gercek = y_test[:10].reset_index(drop=True)
tahmin = pd.Series(y_pred[:10])

# 3. Her biri için duyguları etiket olarak göster
duygu_map = {0: "Negatif", 1: "Pozitif"}

# 4. Hepsini bir tabloya yerleştir
sonuc_df = pd.DataFrame({
    "Tweet": orijinal_tweetler,
    "Gerçek Duygu": gercek.map(duygu_map),
    "Tahmin Edilen Duygu": tahmin.map(duygu_map)
})

# 5. Tabloyu göster
sonuc_df


,Tweet,Gerçek Duygu,Tahmin Edilen Duygu
0,ahhh hope ok,Negatif,Pozitif
1,cool tweet apps razr,Negatif,Pozitif
2,know family drama lamehey next time u hang kim...,Negatif,Pozitif
3,school email wont open geography stuff revise ...,Negatif,Negatif
4,upper airway problem,Negatif,Negatif
5,going miss pastor sermon faith,Negatif,Negatif
6,lunchdj come eat,Pozitif,Pozitif
7,oh feeling like,Negatif,Negatif
8,gahh noopeyton need livethis horrible,Negatif,Negatif
9,thank glad like product review bit site enjoy ...,Pozitif,Pozitif
